In [2]:
import kagglehub
import pandas as pd
import os
import cv2

c:\Users\stani\Documents\Code\Python\HIN\UN-number-detection\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
distribution = [('train', 0.8), ('test', 0.1), ('val', 0.1)]

In [4]:
path = kagglehub.dataset_download("stanislavlevendeev/hazmat-detection")
video_directory = os.environ["PATH_TO_DATA"]
print("Path to dataset files:", path)
print("Path to video files:", video_directory)

Path to dataset files: C:\Users\stani\.cache\kagglehub\datasets\stanislavlevendeev\hazmat-detection\versions\6
Path to video files: C:/Users/stani/Documents/WagonVideos


In [5]:
df = pd.read_csv(path + "/labels_dataframe.csv")
videos = df["Source"].unique()
videos

array(['1690279852.mp4', '1690281365.mp4', '1692830440.mp4',
       '1690801380.mp4', '1691487366.mp4', '1692787289.mp4',
       '1692875102.mp4', '1692945482.mp4', '1693811855.mp4',
       '1693954819.mp4', '1693805101.mp4', '1693820172.mp4',
       '1691496786.mp4', '1692872075.mp4', '1693308534.mp4',
       '1693308657.mp4', '1693309263.mp4', '1693820241.mp4',
       '1693820504.mp4', '1693820904.mp4', '1696009577.mp4',
       '1696374314.mp4', '1696416413.mp4', '1696441496.mp4'], dtype=object)

In [6]:
available_videos = os.listdir(video_directory)
available_videos = [video for video in available_videos if video.endswith(".mp4")]
available_videos = [video for video in available_videos if video in videos]
available_videos

['1690281365.mp4']

In [25]:
total_frames = df[df["Source"].isin(available_videos)]
total_frames = total_frames[["Source", "Frames"]].drop_duplicates()["Frames"].sum()
total_frames

np.int64(733)

In [26]:
dataset = 0
print(int(distribution[dataset][1]))
switch_frame = distribution[dataset][1] * total_frames.astype(int)
print(switch_frame)

0
586.4


In [27]:
frames_count = 0
labels_dir = os.path.join(path, "yolo", "labels", distribution[dataset][0])
frames_dir = os.path.join(path, "yolo", "frames", distribution[dataset][0])
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(frames_dir, exist_ok=True)
for row in available_videos:
    video_path = video_directory + '/' + row
    if (os.path.exists(video_path) == False):
        print(f"File {video_path} not found")
        continue
    processed_source = row.split('.')[0]
    # Open video file
    cap = cv2.VideoCapture(video_path)
    number_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"File {video_path} has {number_of_frames} frames")
    lables = df[df['Source'] == row]
    # save each frame and .txt file with labels
    frame_idx = 0
    while frame_idx < number_of_frames:
        if(frame_idx + frames_count >= switch_frame):
            dataset += 1
            switch_frame += distribution[dataset][1] * total_frames
            labels_dir = os.path.join(path, "yolo", "labels", distribution[dataset][0])
            frames_dir = os.path.join(path, "yolo", "frames", distribution[dataset][0])
            os.makedirs(labels_dir, exist_ok=True)
            os.makedirs(frames_dir, exist_ok=True)
            print(f"Switched to {distribution[dataset][0]} dataset")
        ret, frame = cap.read()  # Read each frame
        img_height, img_width, _ = frame.shape
        img_height = int(img_height)
        img_width = int(img_width)
        if not ret:
            break  # End of video
        # Save frame
        frame_index = str(frame_idx).zfill(5)
        frame_name = f"{processed_source}_{frame_index}.jpg"
        cv2.imwrite(f"{frames_dir}/{frame_name}", frame)
        label_name = f"{processed_source}_{frame_index}.txt"
        if(frame_idx in lables['Relative Frame'].values):
            # Normalize bounding box coordinates
            row = lables[lables['Relative Frame'] == frame_idx]
            x_center = ((float(row['XTL'].values[0]) + float(row['XBR'].values[0])) / 2) / img_width
            y_center = ((float(row['YTL'].values[0]) + float(row['YBR'].values[0])) / 2) / img_height
            width = (float(row['XBR'].values[0]) - float(row['XTL'].values[0])) / img_width
            height = (float(row['YBR'].values[0]) - float(row['YTL'].values[0])) / img_height
            with open(os.path.join(labels_dir, label_name), "w") as f:
                f.write(f"{0} {x_center} {y_center} {width} {height}\n")
        else:
            if(distribution[dataset][0] != "train"):
                with open(os.path.join(labels_dir, label_name), "w") as f:
                    f.write(f"")
        frame_idx += 1
    frames_count += number_of_frames
    cap.release()

File C:/Users/stani/Documents/WagonVideos/1690281365.mp4 has 733 frames
Switched to test dataset
Switched to val dataset
